# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 04** - **Tópico: Pré-processamento de dados. Desbalanceamento de classes**

<br>

**Objetivo deste notebook**: Explorar estratégias para mitigar o problema de desbalanceamento de classes, complementando o pipeline da aula anterior que realizava pré-processamento dos dados através da imputação de valores e transformação de dados (codificação, discretização, normalização).
<br>

---





##**Predição de 'churn' de clientes de serviço de telecomunicação**

Os dados que utilizaremos neste notebook se referem a uma empresa de telecomunicações fictícia que forneceu serviços de telefone residencial e Internet para clientes na Califórnia. O conjunto de dados possui informações sobre os serviços para os quais cada cliente se inscreveu (telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivos, suporte técnico e streaming de TV e filmes), informações da conta do cliente (há quanto tempo eles são clientes, contrato, forma de pagamento, cobrança sem papel, cobranças mensais e cobranças totais) e informações demográficas sobre os clientes (sexo, faixa etária, se eles têm parceiros e dependentes). Há, ainda, uma coluna chamada 'churn', que indica os clientes que desistiram do contrato do serviço no último mês. O objetivo da tarefa preditiva é identificar o churn (i.e., saída) de clientes a partir das informações coletadas. Os dados podem ser acessados neste [link](https://www.kaggle.com/datasets/blastchar/telco-customer-churn).

Este é um problema de predição que usualmente apresenta **classes desbalanceadas**. Neste notebook vamos adicionar ao pipeline de análise dos dados estratégias para tratar o desbalanceamento de classes.



---



In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais

###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, train_test_split, cross_validate, cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

sns.set()

In [ ]:
#!pip install missingno ## instalando biblioteca para visualizar valores faltantes

In [ ]:
import missingno as msno

In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=10VrzI8mA2wPvkNIDaLzv-IglPU-Febtf")#,na_values="NA")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'Churn'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['Churn'])
plt.title("Distribuição do atributo alvo")
plt.show()

Quais os tipos de atributos nos dados? Os dados possuem valores faltantes?
Podemos responder estas perguntas utilizando o método .info() para um dataframe.

In [ ]:
df.info()

A biblioteca missingno permite visualizar os dados faltantes como uma matriz. Outras opções de visualização estão disponíveis e podem ser consultadas na documentação da biblioteca.

In [ ]:
df.isnull().sum()

O atributo customerID é único para cada instância e no geral não possui valor preditivo. Iremos removê-lo da análise.

In [ ]:
customer_ids = df['customerID']
df = df.drop(['customerID'], axis=1)

In [ ]:
##remove as duplicatas
df=df.drop_duplicates(keep='last')

Como atributos categóricos e numéricos podem demandar pré-processamentos diferentes, vamos separar os respectivos "nomes" em dois vetores distintos, facilitando a manipulação dos dados posteriormente.

In [ ]:
## Separa os atributos em vetores, de acordo com o tipo de dado (categórico ou numérico)
cat_columns=list(df.drop(['Churn'], axis=1).select_dtypes(include=["object"]).columns)
print(cat_columns)

num_columns=list(df.select_dtypes(include=["int64", "float64"]).columns)
print(num_columns)

Inspecionando a distribuição dos dados numéricos e categóricos:

In [ ]:
def dist_plot(df,columns,type='boxplot'):
    plt.figure(figsize=(16, 5))
    for indx, var  in enumerate(columns):
        plt.subplot(1, 3, indx+1)
        if (type=='boxplot'):
          g = sns.boxplot(x=var, data=df,showfliers=True)
        else: 
          if (type=='histogram'):
            g = sns.histplot(x=var, data=df)
    plt.tight_layout()

def count_plot(df,columns):
    plt.figure(figsize=(20, 12))
    for indx, var  in enumerate(columns):
        plt.subplot(6, 3, indx+1)
        g = sns.countplot(x=var, data=df)
    plt.tight_layout()

In [ ]:
count_plot(df,cat_columns)

In [ ]:
dist_plot(df,num_columns)#,type="histogram")


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar as classes Yes/No para 1/0.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['Churn'], axis=1)
y = df['Churn'].values

Faremos o mapeamento das classes Yes/No para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui No' por 0, 'Yes' por 1
y = np.array([0 if y=='No' else 1 for y in y]) 

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


## Revisando o pré-processamento e treinamento com Pipelines

Nesta seção, vamos aplicar os pré-processamentos discutidos na aula anterior:

*   Atributos **numéricos**: imputação de valores faltantes (com o método KNN) e normalização
*   Atributos **categóricos**: imputação de valores faltantes (com a moda), one-hot encoding, e normalização.

O último passo é, então, treinar um modelo. 

Podemos usar um Nested Cross-Validation para realizar otimização de hiperparâmetros e análise de desempenho do modelo, conforme visto no notebook da aula anterior. Entretanto, o processo de Nested CV é mais custoso computacionalmente, demandando mais tempo. Assim, as células a seguir realização apenas um k-fold cross-validation simples com o intuito de comparar os desempenhos com e sem a aplicação de estratégias para ajustar o desbalanceamento de dados, mantendo-se fixo o algoritmo de aprendizado supervisionado a ser usado. Entretanto, em projetos e aplicações reais, recomenda-se o uso de nested cross-validation para otimização dos modelos, conforme já foi discutido em aula,


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn import metrics
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC


## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
    ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

## pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos
my_pipeline = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler()),
                 ('classifier',SVC(kernel='rbf', C=0.1))])

# avalia o pipeline
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1) ## usaremos o mesmo nas demais comparações
scores = cross_val_score(my_pipeline, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)

In [ ]:
print('Média F1: %.3f' % np.mean(scores))
print('Desvio Padrão F1: %.3f' % np.std(scores))

---


## Ajustando o desbalanceamento de classes com SMOTE

Nesta seção vamos repetir o experimento anterior, mas utilizando o SMOTE para ajustar o desbalanceamento de classes. O Pipeline padrão do sklearn não suporta o uso de método da blblioteca imblearn. Portanto, importamos também a implementação de Pipeline desta biblioteca (no código referido como imbpipeline)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbpipeline


## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
    ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])



## pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos, seguida por SMOTE (oversampling)
my_pipeline2 = imbpipeline(steps = [['data_transform', data_pipeline],
                                    ['scaler', MinMaxScaler()],
                                    ['smote', SMOTE(random_state=11)],
                                    ['classifier', SVC(kernel='rbf', C=0.1)]])


# avalia o pipeline
scores2 = cross_val_score(my_pipeline2, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)


In [ ]:
print('Média F1: %.3f' % np.mean(scores2))
print('Desvio Padrão F1: %.3f' % np.std(scores2))

---


## Ajustando o desbalanceamento de classes com Undersampling

Nesta seção utilizaremos a abordagem de realizar undersampling a partir da classe majoritária. Na biblioteca imblearn, o método RandomUnderSampler implementa esta estratégia.

In [ ]:
from imblearn.under_sampling import RandomUnderSampler 

## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
    ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])


## pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos, seguida por undersampling
my_pipeline3 = imbpipeline(steps = [['data_transform', data_pipeline],
                                    ['scaler', MinMaxScaler()],
                                    ['under', RandomUnderSampler(random_state=11)],
                                    ['classifier', SVC(kernel='rbf', C=0.1)]])


# avalia o pipeline
scores3 = cross_val_score(my_pipeline3, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)


In [ ]:
print('Média F1: %.3f' % np.mean(scores3))
print('Desvio Padrão F1: %.3f' % np.std(scores3))

---


## Ajustando o desbalanceamento de classes com SMOTE+Tomek links

A abordagem de realizar oversampling com SMOTE e remover instâncias da classe majoritária com o Tomek links também é implementada na biblioteca imblearn.

In [ ]:
from imblearn.combine import SMOTETomek 

## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
    ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])



## pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos, seguida por SMOTE+TOMEK (over + undersampling)
my_pipeline4 = imbpipeline(steps = [['data_transform', data_pipeline],
                                    ['scaler', MinMaxScaler()],
                                    ['smotetomek', SMOTETomek(random_state=11)],
                                    ['classifier', SVC(kernel='rbf', C=0.1)]])


# avalia o pipeline
scores4 = cross_val_score(my_pipeline4, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)


In [ ]:
print('Média F1: %.3f' % np.mean(scores4))
print('Desvio Padrão F1: %.3f' % np.std(scores4))

### Comparando os resultados com amostragem de dados

In [ ]:
results=[]
results.append(scores)
results.append(scores2)
results.append(scores3)
results.append(scores4)
plt.boxplot(results, labels=['Original','Smote','UnderSamp','SmoteTomek'], showmeans=True)
plt.show()

---


## Ajustando o desbalanceamento de classes com cost-sensitive learning

Nesta seção, vamos explorar a abordagem de cost-sensitive learning, ajustando o parâmetro class_weight do método SVC().

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn import metrics
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVC


## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
    ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])

## pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos
my_pipeline5 = Pipeline([
                 ('data_transform', data_pipeline),
                 ('data_normalize',MinMaxScaler()),
                 ('classifier',SVC(kernel='rbf', C=0.1))])


# Set of parameters for which to assess model performances
param_grid = {'classifier__class_weight':[{0: w} for w in [0.01, 0.05, 0.1, 0.5, 1]]}

# avalia o pipeline

# define a estratégia de busca dos melhores hiperparâmetros
search = GridSearchCV(my_pipeline5, param_grid, scoring='f1', n_jobs=1, cv=cv, refit=True)

search.fit(X_train, y_train)

In [ ]:
print('Média F1: %.3f' % np.mean(search.cv_results_["mean_test_score"]))
print('Desvio Padrão F1: %.3f' % np.std(search.cv_results_["mean_test_score"]))

In [ ]:
print(search.best_params_)

---


## Ajustando o desbalanceamento de classes com métodos ensemble

Diversas estratégias de aprendizado ensemble foram adaptadas a fim de lidar com problemas de classificação que apresentam desbalanceamento de classes. Abaixo vamos exemplificar o uso do método BalancedRandomForest. Perceba que neste caso, não estamos usando o SVC, pois o próprio algoritmo de aprendizado supervisionado adaptado faz o  tratamento do desbalanceamento.

In [ ]:
from imblearn.ensemble import BalancedRandomForestClassifier

## pipeline específico para os atributos numéricos
num_pipeline = Pipeline([
    ('imputer', KNNImputer(weights="uniform"))])

## pipeline específico para os atributos categóricos
cat_pipeline = Pipeline([
                         ('imputer', SimpleImputer(strategy='most_frequent')),
                         ('encoder', OneHotEncoder(drop='if_binary', sparse=False))])

## ColumnTransformer para aplicar cada pipeline ao respectivo tipo de atributo
data_pipeline = ColumnTransformer([
                                   ('numerical', num_pipeline, num_columns),
                                   ('categorical', cat_pipeline, cat_columns)])



## pipeline que une as transformações definidas anteriormente e aplica a 
## normalização em todos os atributos, seguida por BalancedRF
my_pipeline6 = imbpipeline(steps = [['data_transform', data_pipeline],
                                    ['scaler', MinMaxScaler()],
                                    ['balancedRF', BalancedRandomForestClassifier(sampling_strategy=1, max_depth=8,random_state=11)]])


# avalia o pipeline
scores6 = cross_val_score(my_pipeline6, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)

In [ ]:
print('Média F1: %.3f' % np.mean(scores6))
print('Desvio Padrão F1: %.3f' % np.std(scores6))


---

## Sua Vez!


A atividade autônoma para reforço do conteúdo segue a mesma linha do que foi proposto no notebook anterior, agregando a correção de desbalanceamento de dados (se pertinente). **Relembrando**, os alunos devem:

*   Se organizar em grupos de **até 3 alunos** (interessante que já seja o grupo para realização do projeto final da disciplina).
*   Escolher um conjunto de dados que **não está pronto** para análise por algoritmos de aprendizado supervisionado, apresentando a **necessidade de limpeza e/ou transformação de dados**. Idealmente, selecionar um conjunto de dados que contenha tipos mistos de atributos e que seja de interesse do grupo para realização do projeto final
*   Implementar um pipeline (recomendado o uso de `Pipeline` no scikit-learn) para realizar o pré-processamento de dados utilizando os métodos vistos nesta semana: imputação de valores, codificação de dados categóricos, discretização de dados numéricos, normalização de dados, ajuste de desbalanceamento de classes. Eventualmente nem todos serão necessários nos dados, mas sugere-se tentar usar ao menos dois destes. A imputação de valores pode ser utilizada tanto para corrigir valores faltantes como ruídos/outliers.
*   Incluir no pipeline o treinamento de modelos de aprendizado de máquina. Os grupos podem optar por realizar um spot-checking de algoritmos e/ou selecionar um algoritmo específico e realizar a otimização dos seus hiperparâmetros, conforme exemplos deste notebook.
*    Executar o pipeline para os dados selecionados, aplicando as etapas de pré-processamento e treinamento de modelos definidas pelo grupo. 
*    Avaliar o desempenho do modelo final com os dados de teste (evitem data leakage!), reportando matriz de confusão e métrica(s) de desempenho(s) selecionada(s) pelo grupo.

Os grupos devem submeter no Moodle o seu notebook do Google Colab exportado em .pdf e .ypnb, devidamente identificado com os nomes dos integrantes do grupo. Sugere-se que, se possível, os grupos incluam no notebook a ser enviado o link para o Google Colab na nuvem, com saídas salvas, a fim de que a professora e monitores(as) possam acessá-lo em caso de dúvidas na avaliação.
